In [5]:
import os
import requests
import shutil

# PASSer API URL
api_url = 'https://passer.smu.edu/api'

# Directories
alloFusion_prediction_set_dir = 'AlloFusion_prediction_set'
passer_ensemble_dir = 'passer_ensemble_prediction_set'
passer_automl_dir = 'passer_automl_prediction_set'
passer_rank_dir = 'passer_rank_prediction_set'

# Create the passer directories if they don't exist
for dir_name in [passer_ensemble_dir, passer_rank_dir]:
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

# Function to query PASSer API and get AFR residues for different models
def get_passer_prediction(pdb_code, chain_name, model):
    data = {"pdb": pdb_code, "chain": chain_name, "model": model}
    response = requests.post(api_url, data=data)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: Could not retrieve results for {pdb_code} chain {chain_name} using model {model}")
        return None

# Function to generate .txt content for AFR (Prediction 1 only)
def generate_txt_content(prediction_data):
    txt_content = "PASSer Allosteric Site Forming Residues (Top Prediction):\n\n"
    # Process only the first prediction ("1")
    if "1" in prediction_data:
        value = prediction_data["1"]
        txt_content += f"Top Prediction: {value['prob']} probability\n"
        txt_content += f"Residues: {value['residues']}\n\n"
    else:
        txt_content += "No top prediction available.\n"
    return txt_content

# Function to generate .pml content for AFR (Prediction 1 only)
def generate_pml_content(pdb_code, chain_name, prediction_data):
    pml_content = f"# PyMOL script to highlight allosteric sites in {pdb_code}\n"
    pml_content += f"fetch {pdb_code}\n"
    pml_content += "hide everything\n"
    pml_content += f"show cartoon, chain {chain_name}\n"
    pml_content += f"color spectrum, chain {chain_name}\n"
    
    # Process only the first prediction ("1")
    if "1" in prediction_data:
        value = prediction_data["1"]
        residues = value['residues'].split('resid ')[-1]
        for res_num in residues.split():
            pml_content += f"select resi {res_num} and chain {chain_name}\n"
            pml_content += f"show surface, resi {res_num} and chain {chain_name}\n"
            pml_content += f"color green, resi {res_num} and chain {chain_name}\n"
            pml_content += f"set transparency, 0.2, resi {res_num} and chain {chain_name}\n"
    
    pml_content += f"zoom chain {chain_name}\n"
    return pml_content

# Automatically build pdb_chain_list from the subfolder names in AlloFusion_prediction_set
pdb_chain_list = []
for folder in os.listdir(alloFusion_prediction_set_dir): 
    pdb_code, chain_name= folder.split('_')
    pdb_chain_list.append({"pdb": pdb_code, "chain": chain_name})

# Loop over each PDB code and chain for all three models
for pdb_chain in pdb_chain_list:
    pdb_code = pdb_chain["pdb"]
    chain_name = pdb_chain["chain"]
    
    # Query PASSer API for the ensemble, automl, and rank models
    for model, dir_name in zip(['ensemble', 'automl', 'rank'], 
                               [passer_ensemble_dir, passer_automl_dir,passer_rank_dir]):
        prediction_data = get_passer_prediction(pdb_code, chain_name, model)
        
        if prediction_data:
            # Create a folder for the protein and chain in the corresponding passer_set directory
            protein_folder = f"{pdb_code}_{chain_name}_passer_{model}"
            target_protein_dir = os.path.join(dir_name, protein_folder)
            
            if not os.path.exists(target_protein_dir):
                os.makedirs(target_protein_dir)
            
            # Copy the two PDB files (protein and chain-specific PDB) from stingallo_prediction_set
            protein_pdb_file = os.path.join(alloFusion_prediction_set_dir, f"{pdb_code}_{chain_name}", f"{pdb_code}_protein.pdb")
            chain_pdb_file = os.path.join(alloFusion_prediction_set_dir, f"{pdb_code}_{chain_name}", f"{pdb_code}_chain_{chain_name}.pdb")
            
            # Check if both files exist before copying
            if os.path.exists(protein_pdb_file):
                shutil.copy(protein_pdb_file, target_protein_dir)
            else:
                print(f"Warning: {protein_pdb_file} does not exist!")
            
            # if os.path.exists(chain_pdb_file):
            #     shutil.copy(chain_pdb_file, target_protein_dir)
            # else:
            #     print(f"Warning: {chain_pdb_file} does not exist!")
            
            # Generate and save the .txt file
            txt_content = generate_txt_content(prediction_data)
            txt_file_path = os.path.join(target_protein_dir, f"{pdb_code}_passer_{model}_allosteric_residues.txt")
            with open(txt_file_path, 'w') as f:
                f.write(txt_content)
            
            # Generate and save the .pml file
            pml_content = generate_pml_content(pdb_code, chain_name, prediction_data)
            pml_file_path = os.path.join(target_protein_dir, f"{pdb_code}_passer_{model}_allosteric_sites.pml")
            with open(pml_file_path, 'w') as f:
                f.write(pml_content)

print("PASSer predictions for ensemble, automl, and rank models created successfully!")

PASSer predictions for ensemble, automl, and rank models created successfully!


In [ ]:
# The following code is for generating .pml files for the AlloFusion prediction set
import os
import re
stingallo_pred_dir = 'alloFusion_prediction_set'
# Function to generate .pml content for AFR (Prediction 1 only)
def generate_pml_content(pdb_code, chain_name, prediction_data):
    pml_content = f"# PyMOL script to highlight allosteric sites in {pdb_code}\n"
    pml_content += f"fetch {pdb_code}\n"
    pml_content += "hide everything\n"
    pml_content += f"show cartoon, chain {chain_name}\n"
    pml_content += f"color spectrum, chain {chain_name}\n"
    
    # Process only the first prediction ("1")
    for res_num in prediction_data:
        pml_content += f"select resi {res_num} and chain {chain_name}\n"
        pml_content += f"show surface, resi {res_num} and chain {chain_name}\n"
        pml_content += f"color red, resi {res_num} and chain {chain_name}\n"
        pml_content += f"set transparency, 0.2, resi {res_num} and chain {chain_name}\n"
    
    pml_content += f"zoom chain {chain_name}\n"
    return pml_content

# read file and extract the prediction data
def get_prediction(pdb_code, chain_name):
    dir_name = stingallo_pred_dir
    txt_file_path = os.path.join(dir_name, f"{pdb_code}_{chain_name}", f"{pdb_code}_allosteric_residues.txt")
    residues = []
    if os.path.exists(txt_file_path):
        with open(txt_file_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                residues = re.findall(r'-?\d+', line)
    return residues

# Automatically build pdb_chain_list from the subfolder names in AlloFusion_prediction_set
pdb_chain_list = [{'pdb':'6E3U','chain':'B'}]
# for folder in os.listdir(stingallo_pred_dir): 
#     pdb_code, chain_name= folder.split('_')
#     pdb_chain_list.append({"pdb": pdb_code, "chain": chain_name})

# Loop over each PDB code and chain for all three models
for pdb_chain in pdb_chain_list:
    pdb_code = pdb_chain["pdb"]
    chain_name = pdb_chain["chain"]
    prediction_data = get_prediction(pdb_code, chain_name)
    print(prediction_data)
    pml_content = generate_pml_content(pdb_code, chain_name, prediction_data)
    target_protein_dir = os.path.join(stingallo_pred_dir, f"{pdb_code}_{chain_name}")
    pml_file_path = os.path.join(target_protein_dir, f"{pdb_code}_allosteric_sites.pml")
    with open(pml_file_path, 'w') as f:
        f.write(pml_content)

['51', '97', '98', '129', '244', '246', '250', '252', '275', '279', '287', '290', '291', '294', '300', '302', '305', '307', '317', '319', '321', '335', '337', '339']


In [ ]:


import os
import numpy as np
import pandas as pd
from Bio.PDB import PDBParser
import warnings
import re

# Directories for the actual and prediction sets
actual_afr_dir = 'actual_afr_set'
alloFusion_pred_dir = 'alloFusion_prediction_set'
passer_ensemble_dir = 'passer_ensemble_prediction_set'
passer_automl_dir = 'passer_automl_prediction_set'
passer_rank_dir = 'passer_rank_prediction_set'
passer_allositePro_dir = 'allositePro_prediction_set'
deepAllo_pred_dir = 'DeepAllo_prediction_set'

dcc_result_file = 'dccPlus_results.csv'
success_rate_file = 'successPlus_rate.txt'

# Initialize PDB parser
parser = PDBParser(QUIET=True)  # Suppress warnings inside the parser

# Function to get the centroid of a list of residues
def calculate_centroid(residues,actual_centroid=None):
    coords = [atom.get_coord() for residue in residues for atom in residue.get_atoms() if atom.get_name() == 'CA']
    coords1=[]
    if coords:
        if actual_centroid is not None:
            for coord in coords:
                if np.linalg.norm(coord-actual_centroid)<=100:
                    coords1.append(coord)
            if coords1:  # Ensure coords1 is not empty
                centroid = np.mean(coords1, axis=0)
                return centroid
            else:
                return None  # Return None if coords1 is empty
        else:
            centroid = np.mean(coords, axis=0)
            return centroid
    return None

# Function to extract residues from a PDB file based on residue numbers
def extract_residues(pdb_file, chain_id, residue_numbers):
    try:
        structure = parser.get_structure('protein', pdb_file)
        chain = structure[0][chain_id]
        residues = [chain[res_id] for res_id in residue_numbers if res_id in chain]
        return residues
    except Exception as e:
        print(f"Error extracting residues from {pdb_file}, Chain {chain_id}: {e}")
        return []

# Function to calculate DCC for predicted vs actual
def calculate_dcc(actual_residues, predicted_residues):
    if not actual_residues or not predicted_residues:
        return None  # Handle cases where no AFR exists or missing residues
    actual_centroid = calculate_centroid(actual_residues)
    predicted_centroid = calculate_centroid(predicted_residues,actual_centroid)
    if actual_centroid is None or predicted_centroid is None:
        return None  # No valid centroids to calculate distance
    
    distance = np.linalg.norm(actual_centroid - predicted_centroid)
    return distance

# Helper function to extract residue numbers from the .pml file
def get_residue_numbers_from_pml(pml_file_path):
    residue_numbers = []
    if os.path.exists(pml_file_path):
        with open(pml_file_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                # Look for lines with 'resi' followed by a number
                match = re.findall(r'\bresi (\d+)', line)
                if match:
                    residue_numbers.extend([int(res_num) for res_num in match])
    return list(set(residue_numbers))

# Function to calculate metrics (EPR, SR) based on true and predicted residues
def calculate_metrics(true_residues, predicted_residues):
    tp = len(set(true_residues) & set(predicted_residues))
    fp = len(set(predicted_residues) - set(true_residues))  # False Positive
    fn = len(set(true_residues) - set(predicted_residues))  # False Negative
   
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    sr=tp/len(true_residues)
    sr=round(sr,2)
    epr=0
    # （EPR）
    if tp>=1:
        epr=1
    else:
        epr=0
    return epr

# Function to process a single set and calculate DCC
def process_set(actual_afr_dir, pred_dir, set_name):
    dcc_data = []
    success_count = 0
    total_count = 0
    
    for folder in os.listdir(actual_afr_dir):
        if folder.endswith('_target'):  # Match actual target folders
            pdb_code, chain = folder.split('_')[:2]
            actual_pdb_file = os.path.join(actual_afr_dir, folder, f'{pdb_code}.pdb')
            # print(actual_pdb_file)
            if set_name == "alloFusion":
                pred_pdb_file = os.path.join(alloFusion_pred_dir, f'{pdb_code}_{chain}', f'{pdb_code}_protein.pdb')
                pml_file_path = os.path.join(alloFusion_pred_dir, f'{pdb_code}_{chain}', f'{pdb_code}_allosteric_sites.pml')
            elif set_name == "passer_ensemble":
                pred_pdb_file = os.path.join(passer_ensemble_dir, f'{pdb_code}_{chain}_passer_ensemble', f'{pdb_code}_protein.pdb')
                pml_file_path = os.path.join(passer_ensemble_dir, f'{pdb_code}_{chain}_passer_ensemble', f'{pdb_code}_passer_ensemble_allosteric_sites.pml')
            elif set_name == "AllositePro":
                pred_pdb_file = os.path.join(passer_allositePro_dir, f'{pdb_code}_{chain}', f'{pdb_code}.pdb')
                pml_file_path = os.path.join(passer_allositePro_dir, f'{pdb_code}_{chain}', f'{pdb_code}_allosteric_sites.pml')
            elif set_name == "passer_rank":
                pred_pdb_file = os.path.join(passer_rank_dir, f'{pdb_code}_{chain}_passer_rank', f'{pdb_code}_protein.pdb')
                pml_file_path = os.path.join(passer_rank_dir, f'{pdb_code}_{chain}_passer_rank', f'{pdb_code}_passer_rank_allosteric_sites.pml')
            elif set_name == "passer_automl":
                pred_pdb_file = os.path.join(passer_automl_dir, f'{pdb_code}_{chain}_passer_automl', f'{pdb_code}_protein.pdb')
                pml_file_path = os.path.join(passer_automl_dir, f'{pdb_code}_{chain}_passer_automl', f'{pdb_code}_passer_automl_allosteric_sites.pml')
            elif set_name == "deepAllo":
                pred_pdb_file = os.path.join(deepAllo_pred_dir, f'{pdb_code}_{chain}', f'{pdb_code}_protein.pdb')
                pml_file_path = os.path.join(deepAllo_pred_dir, f'{pdb_code}_{chain}', f'{pdb_code}_allosteric_sites.pml')
            else:
                continue  # Unknown set

            if not os.path.exists(pred_pdb_file) or not os.path.exists(pml_file_path):
                print(f"Prediction file or PML file not found for {pdb_code} Chain {chain} in {set_name}. Skipping...")
                continue  # Skip if the prediction file or PML file doesn't exist
            
            # Get the list of actual and predicted residue numbers (from the .pml files)
            actual_residue_numbers = get_residue_numbers_from_pml(os.path.join(actual_afr_dir, folder, f'{pdb_code}_allosteric_sites.pml'))
            pred_residue_numbers = get_residue_numbers_from_pml(pml_file_path)
           
            if not actual_residue_numbers or not pred_residue_numbers:
                print(f"No AFRs found for {pdb_code} Chain {chain} in {set_name}. Skipping...")
                continue  # Skip if no AFRs found in either actual or predicted
            
        
             # Extract residues from PDB files
            actual_residues = extract_residues(actual_pdb_file, chain, actual_residue_numbers)
            predicted_residues = extract_residues(pred_pdb_file, chain, pred_residue_numbers)

             # Calculate DCC
            dcc = calculate_dcc(actual_residues, predicted_residues)
            if dcc is not None:
                dcc_data.append([pdb_code, chain, dcc, set_name])
            
            # Increment counters for success rate
            epr=calculate_metrics(actual_residue_numbers, pred_residue_numbers)
            # if epr is not None:
            #     dcc_data.append([pdb_code, chain, epr, set_name])
            if epr is not None and epr == 1:
                success_count += 1
            total_count += 1
    
    return dcc_data, success_count, total_count

# Process the four sets: stingallo, passer_ensemble, passer_automl, passer_rank
dcc_data = []

# Process alloFusion set
alloFusion_dcc_data, stingallo_success, stingallo_total = process_set(actual_afr_dir, alloFusion_pred_dir, 'alloFusion')

# Process passer_ensemble set
passer_ensemble_dcc_data, passer_ensemble_success, passer_ensemble_total = process_set(actual_afr_dir, passer_ensemble_dir, 'passer_ensemble')

# Process allositePro set
passer_allositePro_dcc_data, passer_allositePro_success, passer_allositePro_total = process_set(actual_afr_dir, passer_allositePro_dir, 'AllositePro')

# Process passer_rank set
passer_rank_dcc_data, passer_rank_success, passer_rank_total = process_set(actual_afr_dir, passer_rank_dir, 'passer_rank')

# Process passer_automl set
passer_automl_dcc_data, passer_automl_success, passer_automl_total = process_set(actual_afr_dir, passer_automl_dir, 'passer_automl')

# Process deepAllo set
deepAllo_dcc_data, deepAllo_success, deepAllo_total = process_set(actual_afr_dir, deepAllo_pred_dir, 'deepAllo')
print(deepAllo_success,deepAllo_total)
# Merge DCC results for all sets into a single output with 5 columns
dcc_result = {}
for pdb_code, chain, dcc_stingallo, _ in alloFusion_dcc_data:
    dcc_result[(pdb_code, chain)] = {'DCC_alloFusion': dcc_stingallo}

for pdb_code, chain, dcc_ensemble, _ in passer_ensemble_dcc_data:
    if (pdb_code, chain) in dcc_result:
        dcc_result[(pdb_code, chain)]['DCC_passer_ensemble'] = dcc_ensemble
    else:
        dcc_result[(pdb_code, chain)] = {'DCC_passer_ensemble': dcc_ensemble}

for pdb_code, chain, dcc_allositePro, _ in passer_allositePro_dcc_data:
    if (pdb_code, chain) in dcc_result:
        dcc_result[(pdb_code, chain)]['DCC_AllositePro'] = dcc_allositePro
    else:
        dcc_result[(pdb_code, chain)] = {'DCC_AllositePro': dcc_allositePro}

for pdb_code, chain, dcc_rank, _ in passer_rank_dcc_data:
    if (pdb_code, chain) in dcc_result:
        dcc_result[(pdb_code, chain)]['DCC_passer_rank'] = dcc_rank
    else:
        dcc_result[(pdb_code, chain)] = {'DCC_passer_rank': dcc_rank}

for pdb_code, chain, dcc_automl, _ in passer_automl_dcc_data:
    if (pdb_code, chain) in dcc_result:
        dcc_result[(pdb_code, chain)]['DCC_passer_automl'] = dcc_automl
    else:
        dcc_result[(pdb_code, chain)] = {'DCC_passer_automl': dcc_automl}
for pdb_code, chain, dcc_deepAllo, _ in deepAllo_dcc_data:
    if (pdb_code, chain) in dcc_result:
        dcc_result[(pdb_code, chain)]['DCC_deepAllo'] = dcc_deepAllo
    else:
        dcc_result[(pdb_code, chain)] = {'DCC_deepAllo': dcc_deepAllo}

# Create a DataFrame with the results
dcc_result_list = []
for (pdb_code, chain), dcc_values in dcc_result.items():
    dcc_alloFusion = dcc_values.get('DCC_alloFusion', 'N/A')
    dcc_passer_allositePro = dcc_values.get('DCC_AllositePro', 'N/A')
    dcc_passer_rank = dcc_values.get('DCC_passer_rank', 'N/A')
    dcc_passer_ensemble = dcc_values.get('DCC_passer_ensemble', 'N/A')
    dcc_passer_automl = dcc_values.get('DCC_passer_automl', 'N/A')
    dcc_deepAllo = dcc_values.get('DCC_deepAllo', 'N/A')
    dcc_result_list.append([pdb_code, chain, dcc_alloFusion,  dcc_passer_allositePro, dcc_passer_rank,dcc_passer_ensemble,dcc_passer_automl, dcc_deepAllo])

dcc_df = pd.DataFrame(dcc_result_list, columns=['PDB_Code', 'Chain', 'DCC_alloFusion', 'DCC_AllositesitePro', 'DCC_passer_rank','DCC_passer_ensemble','DCC_passer_automl','DCC_deepAllo'])
dcc_df.to_csv(dcc_result_file, index=False)

# Calculate success rates
alloFusion_success_rate = stingallo_success / stingallo_total if stingallo_total > 0 else 0
passer_ensemble_success_rate = passer_ensemble_success / passer_ensemble_total if passer_ensemble_total > 0 else 0
passer_allositePro_success_rate = passer_allositePro_success / passer_allositePro_total if passer_allositePro_total > 0 else 0
passer_rank_success_rate = passer_rank_success / passer_rank_total if passer_rank_total > 0 else 0
passer_automl_success_rate = passer_automl_success / passer_automl_total if passer_automl_total > 0 else 0
deepAllo_success_rate = deepAllo_success / deepAllo_total if deepAllo_total > 0 else 0
# Write success rates to file
with open(success_rate_file, 'w') as f:
    f.write(f"alloFusion Success Rate: {alloFusion_success_rate:.2%}\n")
    f.write(f"Passer allositePro Success Rate: {passer_allositePro_success_rate:.2%}\n")
    f.write(f"Passer Rank Success Rate: {passer_rank_success_rate:.2%}\n")
    f.write(f"Passer Ensemble Success Rate: {passer_ensemble_success_rate:.2%}\n")
    f.write(f"Passer Automl Success Rate: {passer_automl_success_rate:.2%}\n")
    f.write(f"DeepAllo Success Rate: {deepAllo_success_rate:.2%}\n")
print("DCC calculations and success rate evaluation complete.")